In [4]:
# Cell 1: Import necessary libraries and mount Google Drive
import os
import json
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from google.colab import drive, files

drive.mount('/content/drive')

# Define project folder in Drive
drive_folder = '/content/drive/MyDrive/FaceMaskProject'
os.makedirs(drive_folder, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Cell 2: Set up Kaggle API (skip if not needed, but kept for completeness)
!pip install -q kaggle
!apt-get -qq install -y unzip

# Option 1: Upload kaggle.json (if ever needed for other datasets)
# uploaded = files.upload()
# for fn in uploaded.keys():
#     if 'kaggle.json' in fn:
#         os.makedirs('/root/.kaggle', exist_ok=True)
#         shutil.copy(fn, '/root/.kaggle/kaggle.json')
#         !chmod 600 /root/.kaggle/kaggle.json
#         print("Kaggle API key installed!")

In [6]:
# Cell 3: Prepare dataset by splitting if needed (using existing unsplit version)
unsplit_path = '/content/drive/MyDrive/face_mask_project/Face Mask Dataset'
split_dataset_path = os.path.join(drive_folder, 'Split Face Mask Dataset')

if not os.path.exists(os.path.join(split_dataset_path, 'Train')):
    print("No splits found. Creating Train/Val/Test splits from unsplit dataset.")

    # Define class folders (adjust to your lowercase underscores)
    with_mask_unsplit = os.path.join(unsplit_path, 'with_mask')
    without_mask_unsplit = os.path.join(unsplit_path, 'without_mask')

    if not (os.path.exists(with_mask_unsplit) and os.path.exists(without_mask_unsplit)):
        raise ValueError("Unsplit dataset folders not found. Check path: " + unsplit_path)

    # Get lists of images
    with_mask_images = [os.path.join(with_mask_unsplit, f) for f in os.listdir(with_mask_unsplit) if f.endswith(('.jpg', '.png'))]
    without_mask_images = [os.path.join(without_mask_unsplit, f) for f in os.listdir(without_mask_unsplit) if f.endswith(('.jpg', '.png'))]

    print(f"Found {len(with_mask_images)} with_mask images and {len(without_mask_images)} without_mask images.")

    # Split for with_mask
    train_with, temp_with = train_test_split(with_mask_images, test_size=0.2, random_state=42)
    val_with, test_with = train_test_split(temp_with, test_size=0.5, random_state=42)

    # Split for without_mask
    train_without, temp_without = train_test_split(without_mask_images, test_size=0.2, random_state=42)
    val_without, test_without = train_test_split(temp_without, test_size=0.5, random_state=42)

    # Create split folders with camelcase names
    splits = {
        'Train': {'WithMask': train_with, 'WithoutMask': train_without},
        'Validation': {'WithMask': val_with, 'WithoutMask': val_without},
        'Test': {'WithMask': test_with, 'WithoutMask': test_without}
    }

    os.makedirs(split_dataset_path, exist_ok=True)

    for split_name, classes in splits.items():
        for cls_name, img_list in classes.items():
            cls_path = os.path.join(split_dataset_path, split_name, cls_name)
            os.makedirs(cls_path, exist_ok=True)
            for img in img_list:
                shutil.copy(img, cls_path)

    print("Splits created and saved to Drive:", split_dataset_path)
else:
    print("Using existing split dataset from Drive:", split_dataset_path)

base_path = split_dataset_path  # Use split path for the rest of the code

Using existing split dataset from Drive: /content/drive/MyDrive/FaceMaskProject/Split Face Mask Dataset


In [7]:
# Cell 4: Check dataset folders (run to verify splits)
for split in ['Train', 'Validation', 'Test']:
    for cls in ['WithMask', 'WithoutMask']:
        folder = os.path.join(base_path, split, cls)
        if os.path.exists(folder):
            print(split, cls, ":", len(os.listdir(folder)), "images")
        else:
            print(f"Warning: Folder {folder} not found.")

Train WithMask : 2980 images
Train WithoutMask : 3062 images
Validation WithMask : 372 images
Validation WithoutMask : 383 images
Test WithMask : 373 images
Test WithoutMask : 383 images


In [8]:
# Cell 5: Prepare data generators
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    os.path.join(base_path, 'Train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    os.path.join(base_path, 'Validation'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    os.path.join(base_path, 'Test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 6042 images belonging to 2 classes.
Found 755 images belonging to 2 classes.
Found 756 images belonging to 2 classes.


In [9]:
# Cell 6: Build the model
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base.trainable = False

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
# Cell 7: Train Stage 1 (Feature Extraction)
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(os.path.join(drive_folder, 'face_mask_model_best.keras'),
                                       save_best_only=True, monitor='val_accuracy'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
 18/189 ━━━━━━━━━━━━━━━━━━━━ 12:10 4s/step - accuracy: 0.6133 - loss: 0.6571

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 1124s 6s/step - accuracy: 0.8553 - loss: 0.3311 - val_accuracy: 0.9788 - val_loss: 0.0681
Epoch 2/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 97s 513ms/step - accuracy: 0.9792 - loss: 0.0667 - val_accuracy: 0.9841 - val_loss: 0.0539
Epoch 3/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 96s 510ms/step - accuracy: 0.9844 - loss: 0.0487 - val_accuracy: 0.9828 - val_loss: 0.0504
Epoch 4/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 97s 516ms/step - accuracy: 0.9875 - loss: 0.0368 - val_accuracy: 0.9828 - val_loss: 0.0489
Epoch 5/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 100s 527ms/step - accuracy: 0.9867 - loss: 0.0348 - val_accuracy: 0.9868 - val_loss: 0.0457


In [11]:
# Cell 8: Train Stage 2 (Fine-Tuning)
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 134s 604ms/step - accuracy: 0.9405 - loss: 0.1512 - val_accuracy: 0.9815 - val_loss: 0.0471
Epoch 2/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 101s 534ms/step - accuracy: 0.9776 - loss: 0.0684 - val_accuracy: 0.9868 - val_loss: 0.0365
Epoch 3/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 98s 521ms/step - accuracy: 0.9814 - loss: 0.0443 - val_accuracy: 0.9868 - val_loss: 0.0317
Epoch 4/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 99s 521ms/step - accuracy: 0.9888 - loss: 0.0320 - val_accuracy: 0.9868 - val_loss: 0.0291
Epoch 5/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 99s 525ms/step - accuracy: 0.9864 - loss: 0.0366 - val_accuracy: 0.9894 - val_loss: 0.0312


In [12]:
# Cell 9: Evaluate on Test Set
loss, acc = model.evaluate(test_gen)
print("Test loss:", loss, "Test accuracy:", acc)

test_gen.reset()
preds = model.predict(test_gen, verbose=1)
y_pred = (preds > 0.5).astype(int).reshape(-1)
y_true = test_gen.classes

print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))
print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))

24/24 ━━━━━━━━━━━━━━━━━━━━ 227s 10s/step - accuracy: 0.9899 - loss: 0.0363
Test loss: 0.030530216172337532 Test accuracy: 0.9907407164573669
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 332ms/step
              precision    recall  f1-score   support

    WithMask       0.99      0.99      0.99       373
 WithoutMask       0.99      0.99      0.99       383

    accuracy                           0.99       756
   macro avg       0.99      0.99      0.99       756
weighted avg       0.99      0.99      0.99       756

Confusion matrix:
[[368   5]
 [  2 381]]


In [13]:
# Cell 10: Save final model and class indices
model.save(os.path.join(drive_folder, 'face_mask_model_final.keras'))

class_indices_path = os.path.join(drive_folder, 'face_mask_class_indices.json')
with open(class_indices_path, 'w') as f:
    json.dump(train_gen.class_indices, f)

print("Model and class indices saved to Drive.")

Model and class indices saved to Drive.


In [18]:
# Cell 11: Prediction on uploaded images
with open(class_indices_path) as f:
    idx = json.load(f)
inv_map = {v: k for k, v in idx.items()}

def predict_image(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    x = image.img_to_array(img)
    x = np.expand_dims(x, 0)
    x = preprocess_input(x)
    p = model.predict(x)[0][0]
    label = 'WithMask' if p < 0.5 else 'WithoutMask'
    conf = p if p > 0.5 else (1 - p)
    return label, float(conf)

uploaded = files.upload()
for fn in uploaded.keys():
    lbl, cf = predict_image('/content/' + fn)
    print(fn, "->", lbl, f"(confidence {cf:.2f})")

Saving top-view-male-entrepreneur-suit-posing-camera-isolated-white-surface.jpg to top-view-male-entrepreneur-suit-posing-camera-isolated-white-surface (2).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
top-view-male-entrepreneur-suit-posing-camera-isolated-white-surface (2).jpg -> WithoutMask (confidence 0.97)


In [19]:
# Cell: Zip and download the model and related files
import zipfile
from google.colab import files

# Define paths
project_folder = '/content/drive/MyDrive/FaceMaskProject'
model_path = os.path.join(project_folder, 'face_mask_model_final.keras')
best_model_path = os.path.join(project_folder, 'face_mask_model_best.keras')
class_indices_path = os.path.join(project_folder, 'face_mask_class_indices.json')

# Create a zip file
zip_path = '/content/face_mask_project.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(model_path, arcname='face_mask_model_final.keras')
    if os.path.exists(best_model_path):
        zipf.write(best_model_path, arcname='face_mask_model_best.keras')
    if os.path.exists(class_indices_path):
        zipf.write(class_indices_path, arcname='face_mask_class_indices.json')

print("Files zipped successfully!")

# Download the zip
files.download(zip_path)

Files zipped successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>